#### Consider the linear SDE:

$$ \mathrm{d}X(t) = \mu X(t) \mathrm{d}t + \sigma X(t) \mathrm{d}W(t), X(0) = X_0$$ 

#### where $\mu, \sigma$ are real constants.
#### The exact solution to this SDE is

$$ X(t) = X(0) \exp \left ( (\mu - \frac{1}{2} \sigma^2) t + \sigma W(t) \right )$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(100)

In [ ]:
mu = 1
sigma = 0.1
X_0 = 1
T = 0.1
N = 2**10

dt = float(T) / N
t = np.linspace(0, T, N+1)


In [ ]:
def EM_solver(R, dt, dW, N, X_ref):
    Dt = R * dt
    L = N // R
    X_approx = np.zeros(L + 1)
    X_approx[0] = X_ref[0]

    for j in range(1, L+1):
        W_acc = np.sum(dW[0][range(R*(j-1), R*j)])
        X_approx[j] = X_approx[j-1] + mu * X_approx[j-1] * Dt + sigma * X_approx[j-1] * W_acc

    X_L = X_approx[-1]
    X_T = X_ref[-1]
    
    err = np.abs(X_L - X_T)   
    # err = np.abs(X_approx - X_ref[::R])   
    # print("Error: {:.4e}".format(err))
    
    return err, X_approx, X_L, X_T
    

In [ ]:
def Milstein_solver(R, dt, dW, N, X_ref):
    Dt = R * dt
    L = N // R
    X_approx = np.zeros(L + 1)
    X_approx[0] = X_ref[0]

    for j in range(1, L+1):
        W_acc = np.sum(dW[0][range(R*(j-1), R*j)])
        X_approx[j] = X_approx[j-1] + mu * X_approx[j-1] * Dt + sigma * X_approx[j-1] * W_acc + 0.5 * sigma**2 * X_approx[j-1] * (W_acc**2 - Dt)

    X_L = X_approx[-1]
    X_T = X_ref[-1]
    
    err = np.abs(X_L - X_T)   
    # err = np.abs(X_approx - X_ref[::R])   
    # print("Error: {:.4e}".format(err))
    
    return err, X_approx, X_L, X_T
    

In [ ]:
def RK_solver(R, dt, dW, N, X_ref):
    Dt = R * dt
    L = N // R
    X_approx = np.zeros(L + 1)
    X_approx[0] = X_ref[0]

    for j in range(1, L+1):
        W_acc = np.sum(dW[0][range(R*(j-1), R*j)])
        X_hat = X_approx[j-1] + sigma * X_approx[j-1] * Dt ** 0.5
        X_approx[j] = X_approx[j-1] + mu * X_approx[j-1] * Dt + sigma * X_approx[j-1] * W_acc + 0.5 / Dt ** 0.5 * sigma * (X_hat - X_approx[j-1]) * (W_acc**2 - Dt)

    X_L = X_approx[-1]
    X_T = X_ref[-1]
    
    err = np.abs(X_L - X_T)   
    # err = np.abs(X_approx - X_ref[::R])   
    # print("Error: {:.4e}".format(err))
    
    return err, X_approx, X_L, X_T

In [ ]:
def order(R, X_0 = X_0, dt = dt, N = N, solver = EM_solver):

    dW = np.sqrt(dt) * np.random.randn(1, N)
    W = np.cumsum(dW)

    X_ref = X_0 * np.exp((mu - 0.5*sigma**2)*t[1:] + sigma*W)
    X_ref = np.insert(X_ref, obj = 0, values = X_0)

    err, X_approx, X_L, X_T = solver(R, dt, dW, N, X_ref)

    return err, X_ref, X_approx, X_L, X_T


In [ ]:
MC = 2
R_1 = 2**3
sol_list = np.zeros((MC, N // R_1 + 1))
ref_list = np.zeros((MC, N // R_1 + 1))
for i in range(MC):
    _, X_ref, X_approx, X_L, X_T = order(R = R_1, X_0 = X_0, dt = dt, N = N, solver = EM_solver)
    sol_list[i, :] = X_approx
    ref_list[i, :] = X_approx
    

In [ ]:
ax = plt.subplot(111)
L_1 = N // R_1
ax.plot(np.linspace(0, T, L_1+1), sol_list[0,:], "r-x")
ax.plot(np.linspace(0, T, L_1+1), ref_list[0,:], "k-")
ax.plot(np.linspace(0, T, L_1+1), sol_list[1,:], "b-x")
ax.plot(np.linspace(0, T, L_1+1), ref_list[1,:], "g-")
ax.grid()
ax.legend(("Euler-Maruyama (path 1)", "ref (path 1)", "Euler-Maruyama (path 2)", "ref (path 2)"), loc = "best")
plt.savefig("./Euler-Maruyama_scheme.pdf")
plt.show()

In [ ]:
MC = 2
R_1 = 2**3
sol_list = np.zeros((MC, N // R_1 + 1))
ref_list = np.zeros((MC, N // R_1 + 1))
for i in range(MC):
    _, X_ref, X_approx, X_L, X_T = order(R = R_1, X_0 = X_0, dt = dt, N = N, solver = Milstein_solver)
    sol_list[i, :] = X_approx
    ref_list[i, :] = X_approx

In [ ]:
ax = plt.subplot(111)
L_1 = N // R_1
ax.plot(np.linspace(0, T, L_1+1), sol_list[0,:], "r-x")
ax.plot(np.linspace(0, T, L_1+1), ref_list[0,:], "k-")
ax.plot(np.linspace(0, T, L_1+1), sol_list[1,:], "b-x")
ax.plot(np.linspace(0, T, L_1+1), ref_list[1,:], "g-")
ax.grid()
ax.legend(("Milstein (path 1)", "ref (path 1)", "Milstein (path 2)", "ref (path 2)"), loc = "best")
plt.savefig("./Milstein_scheme.pdf")
plt.show()

In [ ]:
MC = 2
R_1 = 2**3
sol_list = np.zeros((MC, N // R_1 + 1))
ref_list = np.zeros((MC, N // R_1 + 1))
for i in range(MC):
    _, X_ref, X_approx, X_L, X_T = order(R = R_1, X_0 = X_0, dt = dt, N = N, solver = RK_solver)
    sol_list[i, :] = X_approx
    ref_list[i, :] = X_approx

In [ ]:
ax = plt.subplot(111)
L_1 = N // R_1
ax.plot(np.linspace(0, T, L_1+1), sol_list[0,:], "r-x")
ax.plot(np.linspace(0, T, L_1+1), ref_list[0,:], "k-")
ax.plot(np.linspace(0, T, L_1+1), sol_list[1,:], "b-x")
ax.plot(np.linspace(0, T, L_1+1), ref_list[1,:], "g-")
ax.grid()
ax.legend(("Runge-Kutta (path 1)", "ref (path 1)", "Runge-Kutta (path 2)", "ref (path 2)"), loc = "best")
plt.savefig("./Runge-Kutta_scheme.pdf")
plt.show()